## Library
---

In [127]:
import os
import time
import numpy as np
from ase import Atoms, neighborlist, Atom
from ase.io import write
from ase.lattice.cubic import FaceCenteredCubic

## Algorithm
---

In [2]:
'''
site energy -> activation energy -> diffusion rate

site energy(E_i) = -N*(E_b)/2

activation energy(E_a) = E_0 + alpha * E_r
E_r = E_i(end) - E_i(start)

diffusion rate = f*exp(-E_a/(k*T)) (Arrhenius)
f is ~ 10^13 for most metals
'''

def get_site_energy(bond_energy, bond_num):
    return -bond_num * bond_energy/2

def get_activation_energy(e_start, e_end, alpha = 0.1, e0 = 0):
    e_reaction = e_end-e_start
    if e_reaction>=0:
        return e0 + (1+alpha)*e_reaction
    else:
        return e0 + alpha*e_reaction

def get_diffusion_rate(e_a, T=300, f=1E13):
    k_B = 8.617333262145e-5  # Boltzmann constant in eV/K
    return f*np.exp(-e_a/(k_B*T))

In [229]:
# function to create fcc lattice
def create_fcc_lattice(symbol, lattice_constant, repetitions):
    """
    Create an FCC lattice of atoms.

    Args:
    - symbol: The chemical symbol of the atoms.
    - lattice_constant: The lattice constant of the FCC lattice.
    - repetitions: The number of repetitions of the unit cell in each direction.
                [x, y, z]
    - vacuum: The thickness of the vacuum layer outside the plane perpendicular to the z-axis.

    Returns:
    - An Atoms object representing the FCC lattice.
    """

    # Define the positions of the atoms in the unit cell
    positions = [[0, 0, 0],
                 [0.5, 0.5, 0],
                 [0.5, 0, 0.5],
                 [0, 0.5, 0.5]]

    # Scale the positions by the lattice constant
    positions = [[x * lattice_constant for x in pos] for pos in positions]

    # Create the FCC lattice
    fcc_lattice = Atoms([symbol] * 4, positions=positions, cell=[lattice_constant] * 3, pbc=[True, True, False])

    # Repeat the unit cell in x and y directions
    fcc_lattice *= repetitions

    return fcc_lattice


In [87]:
# function to find the closest neighbors of a vacancy

def get_closest_neighbors(lattice, vacancy_index):
    """
    Get the indices of the closest neighboring atoms to a vacancy in a lattice.

    Args:
    - lattice: The lattice containing the vacancy and neighboring atoms.
    - vacancy_index: The index of the vacancy in the lattice.
    - num_neighbors: The number of neighboring atoms to return.

    Returns:
    - A list of the indices of the closest neighboring atoms to the vacancy.
    """
    # Create a neighbor list
    cutoff = neighborlist.natural_cutoffs(lattice)
    nl = neighborlist.NeighborList(cutoff, self_interaction=False, bothways=True)
    nl.update(lattice)

    # Get the indices of the atoms that are neighbors to the vacancy
    indices, offsets = nl.get_neighbors(vacancy_index)

    return indices



In [237]:
# function to diffusion of a vacancy

def find_candidate(lattice):
    global diffusion_rate

    candidate_table = []    # list of candidate atoms. element: [start_pos, end_pos]
    diffusion_table = []    # list of diffusion rate. Indices corresponding to the candidate_table  element: diffusion rate

    # 1. find a candididate
    vacancy_indices = lattice.symbols.search('X')
    for vac_idx in vacancy_indices:
        lattice[vac_idx].symbol = 'Cu'
        candidate_idx = get_closest_neighbors(lattice, vac_idx)
        lattice[vac_idx].symbol = 'X'
        end_point_bond_num = len(candidate_idx)-1

        # 2. modify event table
        for idx in candidate_idx:
            # position
            candidate_table.append([idx, vac_idx])

            # diffusion rate
            start_point_bond_num = len(get_closest_neighbors(lattice, idx))
            diffusion_table.append(diffusion_rate[start_point_bond_num][end_point_bond_num])
    
    return candidate_table, diffusion_table

def diffuse_one_step(lattice):
    global time_elapsed

    # 1. find a candidate
    candidate_table, diffusion_table = find_candidate(lattice)
    diffusion_table = np.array(diffusion_table)
    total_diffusion_rate = diffusion_table.sum()

    # 2. pick random numbers
    u = np.random.uniform(low=1e-6, high=1)
    u_time = np.random.uniform(low=1e-6, high=1)
    cum_dif = np.cumsum(diffusion_table)
    chosen_idx = np.argwhere(u*total_diffusion_rate < cum_dif)[0][0]
    # print(chosen_idx)

    # 3. time update
    delta_t = -np.log(u_time)/total_diffusion_rate
    time_elapsed += delta_t

    # 4. diffusion(swap the position of atom 'A' and vacancy 'X')
    start_idx = candidate_table[chosen_idx][0]
    end_idx = candidate_table[chosen_idx][1]
    start_pos = lattice[start_idx].position.copy()
    end_pos = lattice[end_idx].position
    
    lattice[start_idx].position = end_pos
    lattice[end_idx].position = start_pos
    

## Parameter setting
---

In [219]:
# parameter for diffusion rate
'''
bond energy : 200kJ/mol ~~ 2.07 eV/particle
fcc -> 12 nearest neighbors
'''
bond_energy = 2.07 
temperature = 500
e0 = 0.1
num_closest_neighbors = 12

# site energy, e_(bond number)
e_site = np.zeros(num_closest_neighbors)
for i in range(num_closest_neighbors):
    e_site[i] = get_site_energy(bond_energy, i)

# activation energy, e_a_(start to end)
e_a = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        e_a[i, j] = get_activation_energy(e_site[i], e_site[j], e0=e0)

# diffusion rate, rate_(start to end)
diffusion_rate = np.zeros((num_closest_neighbors, num_closest_neighbors))
for i in range(num_closest_neighbors):
    for j in range(num_closest_neighbors):
        diffusion_rate[i, j] = get_diffusion_rate(e_a[i, j], temperature)
   

In [221]:
# value check
# print(e_site)
# print('-----------------')
# # print(e_a)
# print('-----------------')
# print(diffusion_rate)

## Simulation
---

In [238]:
# parameter tuning
time_elapsed = 0
save_file_name = 'ovito/result2.xyz'
steps = 1000

# create lattice
lattice = create_fcc_lattice('Cu', 3.6, [10, 10, 3])

# make vacancy
lattice[1000].symbol = 'X'
write(save_file_name, lattice)

# start simulation
for i in range(1, steps+1):
    diffuse_one_step(lattice)
    write(save_file_name, lattice, append=True)
    if i % 10 == 0:
        print(f'---------------- step {i} ---------------------')
        print(f'Time elapsed in simul: {time_elapsed} s')

/tmp/ipykernel_30705/3194264741.py:30: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  fcc_lattice *= repetitions


---------------- step 10 ---------------------
Time elapsed in simul: 3.391397069860454e-12 s
---------------- step 20 ---------------------
Time elapsed in simul: 7.154356049631523e-12 s
---------------- step 30 ---------------------
Time elapsed in simul: 1.0300637548814958e-11 s
---------------- step 40 ---------------------
Time elapsed in simul: 1.2712671076104162e-11 s
---------------- step 50 ---------------------
Time elapsed in simul: 1.5234625373385467e-11 s
---------------- step 60 ---------------------
Time elapsed in simul: 1.725290977059267e-11 s
---------------- step 70 ---------------------
Time elapsed in simul: 1.9647574187238862e-11 s
---------------- step 80 ---------------------
Time elapsed in simul: 2.2211556125913465e-11 s
---------------- step 90 ---------------------
Time elapsed in simul: 2.4390187408132605e-11 s
---------------- step 100 ---------------------
Time elapsed in simul: 2.7979439776462333e-11 s
---------------- step 110 ---------------------
Time

KeyboardInterrupt: 

In [230]:
lattice = create_fcc_lattice('Cu', 3.6, [10, 10, 3])

/tmp/ipykernel_30705/3194264741.py:30: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  fcc_lattice *= repetitions


In [235]:
lattice[1000]
lattice.cell

Cell([36.0, 36.0, 10.8])